In [3]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [4]:
filelist = glob.glob('train/glaucoma_resized/*.jpg')              #change '/'
glu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(glu.shape)

(40, 605, 700, 3)


In [5]:
filelist = glob.glob('train/ng/*.jpg')         #change '/'
nonglu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(nonglu.shape)

(40, 605, 700, 3)


In [6]:
X = np.concatenate((glu,nonglu),axis=0)
print(X.shape)
print(X[1,:].shape)

(80, 605, 700, 3)
(605, 700, 3)


In [7]:
gluL = np.array([1 for i in range(40)])
nongluL = np.array([0 for i in range(40)])
y = np.concatenate((gluL,nongluL),axis=0)
print(y.shape)

(80,)


In [8]:
from sklearn.preprocessing import normalize 
for i in range(80):
    X[i,:,:,0] = normalize(X[i,:,:,0])
    X[i,:,:,1] = normalize(X[i,:,:,1])
    X[i,:,:,2] = normalize(X[i,:,:,2])
    
print(X.shape)

(80, 605, 700, 3)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_test)
print(y_train)

(64, 605, 700, 3) (16, 605, 700, 3) (64,) (16,)
[1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0]
[0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 0 0 0 1
 1 0 1 0 1 0 0 0 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 1 0]


In [26]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same',kernel_initializer='random_uniform',
                bias_initializer='zeros', input_shape=X_train.shape[1:]))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.35))
model.add(Conv2D(32, (3,3),padding='valid',kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.35))

model.add(Conv2D(64, (3,3), strides =2,kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.35))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3,3), strides=2,kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.35))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, (3,3), strides =2,kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.35))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(1024, (1,1), strides =2,kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.35))

model.add(Flatten())
model.add(Dense(32,kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.35))
model.add(Dense(16,kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(Activation('sigmoid'))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [27]:
model.output_shape

(None, 1)

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 605, 700, 32)      896       
_________________________________________________________________
activation_17 (Activation)   (None, 605, 700, 32)      0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 605, 700, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 603, 698, 32)      9248      
_________________________________________________________________
activation_18 (Activation)   (None, 603, 698, 32)      0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 603, 698, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 301, 348, 64)      18496     
__________

In [29]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=0.0003, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train, epochs=128, batch_size=2,validation_data=(X_test, y_test))

Train on 64 samples, validate on 16 samples
Epoch 1/128
64/64 [==============================] - 79s 1s/step - loss: 0.6996 - acc: 0.5469 - val_loss: 0.7374 - val_acc: 0.3125
Epoch 2/128
64/64 [==============================] - 79s 1s/step - loss: 0.7132 - acc: 0.5312 - val_loss: 0.7382 - val_acc: 0.3125
Epoch 3/128
64/64 [==============================] - 80s 1s/step - loss: 0.7551 - acc: 0.4062 - val_loss: 0.7297 - val_acc: 0.3125
Epoch 4/128
64/64 [==============================] - 80s 1s/step - loss: 0.7146 - acc: 0.5000 - val_loss: 0.7306 - val_acc: 0.3125
Epoch 5/128
64/64 [==============================] - 79s 1s/step - loss: 0.7828 - acc: 0.3906 - val_loss: 0.7189 - val_acc: 0.3125
Epoch 6/128
64/64 [==============================] - 80s 1s/step - loss: 0.7129 - acc: 0.5781 - val_loss: 0.7192 - val_acc: 0.3125
Epoch 7/128
64/64 [==============================] - 79s 1s/step - loss: 0.7391 - acc: 0.4844 - val_loss: 0.7284 - val_acc: 0.3125
Epoch 8/128
64/64 [====================

Epoch 63/128
64/64 [==============================] - 80s 1s/step - loss: 0.6995 - acc: 0.5938 - val_loss: 0.7308 - val_acc: 0.3125
Epoch 64/128
64/64 [==============================] - 80s 1s/step - loss: 0.6901 - acc: 0.5312 - val_loss: 0.7351 - val_acc: 0.3125
Epoch 65/128
64/64 [==============================] - 80s 1s/step - loss: 0.7136 - acc: 0.4844 - val_loss: 0.7374 - val_acc: 0.3125
Epoch 66/128
64/64 [==============================] - 79s 1s/step - loss: 0.6671 - acc: 0.5625 - val_loss: 0.7324 - val_acc: 0.3125
Epoch 67/128
64/64 [==============================] - 79s 1s/step - loss: 0.6793 - acc: 0.5625 - val_loss: 0.7334 - val_acc: 0.3125
Epoch 68/128
64/64 [==============================] - 79s 1s/step - loss: 0.6861 - acc: 0.4375 - val_loss: 0.7330 - val_acc: 0.3125
Epoch 69/128
64/64 [==============================] - 79s 1s/step - loss: 0.6911 - acc: 0.5000 - val_loss: 0.7277 - val_acc: 0.3125
Epoch 70/128
64/64 [==============================] - 79s 1s/step - loss: 0.

Epoch 127/128
64/64 [==============================] - 80s 1s/step - loss: 0.6980 - acc: 0.5625 - val_loss: 0.7325 - val_acc: 0.3125
Epoch 128/128
64/64 [==============================] - 79s 1s/step - loss: 0.6921 - acc: 0.5781 - val_loss: 0.7298 - val_acc: 0.3125


In [36]:
score = model.evaluate(X_test, y_test, batch_size=1)
print(score)

16/16 [==============================] - 5s 309ms/step
[0.7297717146575451, 0.3125]


In [37]:
model.predict(X_test, batch_size=1)
model.predict_classes(X_test,batch_size=1)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=int32)

In [38]:
import h5py
model.save_weights('better_weights.h5')